Percobaan ketiga dengan transfer learning CNN menggunakan model DenseNet121 </br>
Training dilakukan dengan 70 epochs </br>
Model yang dihasilkan punya akurasi validasi (test) 94%. Nilai tersebut diperoleh pada epoch ke 67 </br>
History dari training disimpan di file historyModelTransferLearning.h5 </br>
Training dilakukan di platform Google Colab </br>

<h2> ====IMPORT LIBRARY==== </h2>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow
from tensorflow.keras import optimizers, regularizers, Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D, BatchNormalization
from tensorflow.keras.models  import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.applications import densenet
import numpy as np
import os
import time
import glob
import pickle
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


<h2>====DATASET INPUT====</h2>

In [5]:
%cd gdrive/My\ Drive/TubesPemlan/TubesPemlan

/content/gdrive/My Drive/TubesPemlan/TubesPemlan


In [6]:
path = "dataset-resized"
listImage = glob.glob(os.path.join(path, '*/*.jpg'))
print("Jumlah dataset: " + str(len(listImage)))

Jumlah dataset: 2527


<h2> ====IMAGE AUGMENTATION==== </h2>

Membuat image augmentation pada dataset yang sudah ada untuk memperbanyak jumlah dataset</br>
Membagi dataset yang ada menjadi dataset untuk training dan test </br>



In [7]:
train = ImageDataGenerator(horizontal_flip=True,
                          validation_split=0.1,
                          rescale=1./255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          width_shift_range=0.2,
                          height_shift_range=0.2)

test = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = train.flow_from_directory(path, 
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode="categorical",
                                            subset="training")

test_generator = test.flow_from_directory(path,
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode="categorical",
                                          subset="validation")

#label untuk image
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [8]:
for img,label in train_generator:
    break
img.shape, label.shape

((32, 224, 224, 3), (32, 6))

In [ ]:
Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

<h2> ====CNN (Convolutional Neural Network)==== </h2>

Load model dari DenseNet121

In [10]:
base_model = densenet.DenseNet121(include_top=False,
                                  weights="imagenet",
                                  input_tensor=None,
                                  input_shape=(224,224,3),
                                  pooling="max")

x = base_model.output
x = Dense(256, activation="elu", kernel_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.6)(x)
x = BatchNormalization()(x)
predictions = Dense(6, activation="softmax")(x)

model = Model(base_model.input, outputs=predictions)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
29089792/29084464 [==============================] - 1s 0us/step


In [ ]:
#menyimpan model
modelFile = "modelConfigTransferLearning.h5"
checkpoint1 = ModelCheckpoint(modelFile, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
callback_list = [checkpoint1]

In [ ]:
optimizer = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

<h2>====TRAINING====</h2>

In [14]:
%%time
history = model.fit_generator(train_generator,
                              epochs=70,
                              steps_per_epoch=len(train_generator),
                              validation_data=test_generator,
                              validation_steps=len(test_generator),
                              callbacks=callback_list)

Epoch 1/70
 8/72 [==>...........................] - ETA: 18s - loss: 30.9561 - acc: 0.7610
Epoch 00001: val_acc improved from -inf to 0.76096, saving model to modelConfigTransferLearning.h5
72/72 [==============================] - 166s 2s/step - loss: 31.4292 - acc: 0.7685 - val_loss: 30.9561 - val_acc: 0.7610
Epoch 2/70
 8/72 [==>...........................] - ETA: 18s - loss: 30.1322 - acc: 0.7729
Epoch 00002: val_acc improved from 0.76096 to 0.77291, saving model to modelConfigTransferLearning.h5
72/72 [==============================] - 64s 895ms/step - loss: 30.5151 - acc: 0.7781 - val_loss: 30.1322 - val_acc: 0.7729
Epoch 3/70
 8/72 [==>...........................] - ETA: 18s - loss: 29.1036 - acc: 0.8327
Epoch 00003: val_acc improved from 0.77291 to 0.83267, saving model to modelConfigTransferLearning.h5
72/72 [==============================] - 64s 892ms/step - loss: 29.6218 - acc: 0.7931 - val_loss: 29.1036 - val_acc: 0.8327
Epoch 4/70
 8/72 [==>...........................] - ET

In [ ]:
#save history ke file
with open('historyModelTransferLearning', 'wb') as f :
    pickle.dump(history.history, f)

In [ ]:
train_acc = history.history['acc']
validation_acc = history.history['val_acc']

train_loss = history.history['loss']
validation_loss = history.history['val_loss']

#plot grafik akurasi
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(validation_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

#plot grafik loss
plt.subplot(2, 1, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()